# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
# Importando bibliotecas
import pandas as pd
import numpy as np

import patsy
import statsmodels.api as sm

In [3]:
# Importando os dados
link = "https://raw.githubusercontent.com/gfsilveira/ebac/main/"
link += "03-Cientista%20de%20Dados/02-Crisp-DM/M%C3%B3dulo_10/"
link += "Profissao-%20Cientista%20de%20Dados_M10_support%20material.csv"

df = pd.read_csv(link)
df.drop(["Unnamed: 0","index"], axis=1, inplace=True)
df = df.dropna()
df.head()

,data_ref,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,mau,renda
0,2015-01-01,F,False,True,1,Assalariado,Secundário,Casado,Casa,36,3.575342,3.0,False,3369.24
1,2015-01-01,M,True,True,0,Empresário,Secundário,Casado,Casa,42,0.860274,2.0,False,6096.14
2,2015-01-01,M,True,True,2,Assalariado,Superior completo,Casado,Casa,31,8.065753,4.0,False,5658.98
3,2015-01-01,F,True,False,0,Empresário,Secundário,Casado,Casa,50,1.208219,2.0,False,7246.69
4,2015-01-01,M,False,False,0,Assalariado,Secundário,Casado,Casa,52,13.873973,2.0,False,4017.37


In [4]:
# Verificando as informações da DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12466 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               12466 non-null  object 
 1   sexo                   12466 non-null  object 
 2   posse_de_veiculo       12466 non-null  bool   
 3   posse_de_imovel        12466 non-null  bool   
 4   qtd_filhos             12466 non-null  int64  
 5   tipo_renda             12466 non-null  object 
 6   educacao               12466 non-null  object 
 7   estado_civil           12466 non-null  object 
 8   tipo_residencia        12466 non-null  object 
 9   idade                  12466 non-null  int64  
 10  tempo_emprego          12466 non-null  float64
 11  qt_pessoas_residencia  12466 non-null  float64
 12  mau                    12466 non-null  bool   
 13  renda                  12466 non-null  float64
dtypes: bool(3), float64(3), int64(2), object(6)
memory usa

## 1 Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

In [5]:
def mais_frequente(coluna: str) -> None:
  '''
  Função que analisa as categorias na variável recebida, imprimindo a mais
  frequênte
  :param coluna (str): Nome da coluna à ser analisada.
  '''
  print(coluna)
  print(df[coluna].value_counts(normalize=True).index[0])

In [6]:
# Verificando qual valor é mais frequênte em cada variável
for c in range(1,9):
  mais_frequente(coluna=df.columns[c])
  print("-"*10)

sexo
F
----------
posse_de_veiculo
False
----------
posse_de_imovel
True
----------
qtd_filhos
0
----------
tipo_renda
Assalariado
----------
educacao
Secundário
----------
estado_civil
Casado
----------
tipo_residencia
Casa
----------


In [7]:
# Verificando qual valor é mais frequênte em cada variável
mais_frequente(coluna=df.columns[11])

qt_pessoas_residencia
2.0


In [8]:
# Verificando qual valor é mais frequênte em cada variável
mais_frequente(coluna=df.columns[12])

mau
False


In [9]:
# Obtendo o modelo de regressão com as categorias mais frequêntes nas caselas
formula = '''
              np.log(renda) ~
                idade +
                tempo_emprego +
                C(sexo, Treatment('F')) +
                C(posse_de_veiculo, Treatment(False)) +
                C(posse_de_imovel, Treatment(True)) +
                C(qtd_filhos, Treatment(0)) +
                C(tipo_renda, Treatment('Assalariado')) +
                C(educacao, Treatment('Secundário')) +
                C(estado_civil, Treatment('Casado')) +
                C(tipo_residencia, Treatment('Casa')) +
                C(qt_pessoas_residencia, Treatment(2.0)) +
                C(mau, Treatment(False))
          '''
y, x = patsy.dmatrices(formula, df)
modelo = sm.OLS(y, x).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.243
Model:                            OLS   Adj. R-squared:                  0.241
Method:                 Least Squares   F-statistic:                     113.9
Date:                Thu, 28 Mar 2024   Prob (F-statistic):               0.00
Time:                        18:39:46   Log-Likelihood:                -13195.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12430   BIC:                         2.673e+04
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

### Conclusão:

- De modo geral, as variáveis com menor significância são as quantitativas;
- Uma análise de variáveis deve ser feito para verificar a relevância, pois algumas delas não são significativas nesse contexto;



---



## 2 Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

In [10]:
'''
Obtendo o modelo de regressão com as categorias mais frequêntes nas caselas
Retirando as não significativas.
'''
formula = '''
                np.log(renda) ~
                idade +
                tempo_emprego +
                C(sexo, Treatment('F')) +
                C(posse_de_imovel, Treatment(True)) +
                C(tipo_renda, Treatment('Assalariado')) +
                C(educacao, Treatment('Secundário')) +
                C(tipo_residencia, Treatment('Casa'))
          '''
y, x = patsy.dmatrices(formula, df)
modelo = sm.OLS(y, x).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.241
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     232.8
Date:                Thu, 28 Mar 2024   Prob (F-statistic):               0.00
Time:                        18:39:46   Log-Likelihood:                -13209.
No. Observations:               12466   AIC:                         2.645e+04
Df Residuals:                   12448   BIC:                         2.659e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [11]:
'''
Obtendo o modelo de regressão com as categorias mais frequêntes nas caselas
Retirando as não significativas e as quantitativas.
'''
formula = '''
                np.log(renda) ~
                C(sexo, Treatment('F')) +
                C(posse_de_imovel, Treatment(True)) +
                C(tipo_renda, Treatment('Assalariado')) +
                C(educacao, Treatment('Secundário')) +
                C(tipo_residencia, Treatment('Casa'))
          '''
y, x = patsy.dmatrices(formula, df)
modelo = sm.OLS(y, x).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     60.60
Date:                Thu, 28 Mar 2024   Prob (F-statistic):          1.96e-177
Time:                        18:39:46   Log-Likelihood:                -14490.
No. Observations:               12466   AIC:                         2.901e+04
Df Residuals:                   12450   BIC:                         2.913e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

### Conclusão

- Métricas dos modelos:

|Métrica|Todas as variáveis|Todas Variáveis Significativas|Variáveis Significativas Qualitativas|
|-|-|-|-|
|R²      |  0.243  | 0.241  | 0.068  |
|R² ajust|  0.241  | 0.240  | 0.067  |
|AIC     |  26.460 | 26.450 | 29.010 |
<br>

- As variáveis significativas foram:
<br>- Sexo;
<br>- Posse de Imóvel;
<br>- Tipo da Renda;
<br>- Educação;
<br>- Tipo de Residência;


- Removendo as variáveis não significativas, o modelo apresentou métricas sem mudanças significtivas.

- A análise mostra que a presença das variáveis quantitativas é necessário para manter o coeficiente de diferenciação.



---



## 3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

In [12]:
'''
Obtendo o modelo de regressão com as categorias mais frequêntes nas caselas
Retirando o tipo de residência.
'''
formula = '''
                np.log(renda) ~
                idade +
                tempo_emprego +
                C(sexo, Treatment('F')) +
                C(posse_de_imovel, Treatment(True)) +
                C(tipo_renda, Treatment('Assalariado')) +
                C(educacao, Treatment('Secundário'))
          '''
y, x = patsy.dmatrices(formula, df)
modelo = sm.OLS(y, x).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     328.5
Date:                Thu, 28 Mar 2024   Prob (F-statistic):               0.00
Time:                        18:39:47   Log-Likelihood:                -13215.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12453   BIC:                         2.655e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [13]:
'''
Obtendo o modelo de regressão com as categorias mais frequêntes nas caselas
Retirando educação.
'''
formula = '''
                np.log(renda) ~
                idade +
                tempo_emprego +
                C(sexo, Treatment('F')) +
                C(posse_de_imovel, Treatment(True)) +
                C(tipo_renda, Treatment('Assalariado'))
          '''
y, x = patsy.dmatrices(formula, df)
modelo = sm.OLS(y, x).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     481.8
Date:                Thu, 28 Mar 2024   Prob (F-statistic):               0.00
Time:                        18:39:47   Log-Likelihood:                -13249.
No. Observations:               12466   AIC:                         2.652e+04
Df Residuals:                   12457   BIC:                         2.658e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [14]:
'''
Obtendo o modelo de regressão com as categorias mais frequêntes nas caselas
Retirando tipo de renda.
'''
formula = '''
                np.log(renda) ~
                idade +
                tempo_emprego +
                C(sexo, Treatment('F')) +
                C(posse_de_imovel, Treatment(True))
          '''
y, x = patsy.dmatrices(formula, df)
modelo = sm.OLS(y, x).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.225
Method:                 Least Squares   F-statistic:                     907.7
Date:                Thu, 28 Mar 2024   Prob (F-statistic):               0.00
Time:                        18:39:47   Log-Likelihood:                -13335.
No. Observations:               12466   AIC:                         2.668e+04
Df Residuals:                   12461   BIC:                         2.672e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [15]:
'''
Obtendo o modelo de regressão com as categorias mais frequêntes nas caselas
Retirando posse de imóvel.
'''
formula = '''
                np.log(renda) ~
                idade +
                tempo_emprego +
                C(sexo, Treatment('F'))
          '''
y, x = patsy.dmatrices(formula, df)
modelo = sm.OLS(y, x).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     1185.
Date:                Thu, 28 Mar 2024   Prob (F-statistic):               0.00
Time:                        18:39:47   Log-Likelihood:                -13365.
No. Observations:               12466   AIC:                         2.674e+04
Df Residuals:                   12462   BIC:                         2.677e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [16]:
'''
Obtendo o modelo de regressão com as categorias mais frequêntes nas caselas
Retirando sexo.
'''
formula = '''
                np.log(renda) ~
                idade +
                tempo_emprego +
                C(posse_de_imovel, Treatment(True))
          '''
y, x = patsy.dmatrices(formula, df)
modelo = sm.OLS(y, x).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.148
Model:                            OLS   Adj. R-squared:                  0.147
Method:                 Least Squares   F-statistic:                     719.6
Date:                Thu, 28 Mar 2024   Prob (F-statistic):               0.00
Time:                        18:39:47   Log-Likelihood:                -13933.
No. Observations:               12466   AIC:                         2.787e+04
Df Residuals:                   12462   BIC:                         2.790e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

### Conclusão

- O modelo teve uma redução na capacidade explicativa da variável dependente, quando as varíaveis qualitativas foram retiradas.

|Variável|R²|R² ajust| AIC|
|-|-|-|-|
|tipo de residência | 0.240|0.240|2.646e+04|
|educação           | 0.236|0.236|2.652e+04|
|tipo de renda      | 0.226|0.225|2.668e+04|
|posse de imóvel    | 0.222|0.222|2.674e+04|
|sexo               | 0.148|0.147|2.787e+04|
<br>

- A variável com o maior poder explicativo parece ser sexo, seguido de posse de imóvel;
- Já o R² ajustado segue o mesmo padrão de redução do R², mostrando que a retirada das variévis têm pouco impacto nessa métrica.
- O AIC aumentou, à medida que variáveis foram retiradas, mostrando sua relevância;
- O modelo mais equilibrado seria o que utiliza:
<br>-idade
<br>-tempo_emprego
<br>-sexo (casela F)
<br>-posse_de_imovel (casela True)
<br>-tipo_renda (casel Assalariado